In [30]:
!nvidia-smi


Tue Mar 28 22:03:20 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.41                 Driver Version: 531.41       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti    WDDM | 00000000:01:00.0 Off |                  N/A |
| N/A   50C    P8                5W /  N/A|    410MiB /  4096MiB |     11%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [31]:
import tensorflow as tf
print(tf.__version__)

2.11.0


#Importing Libraries

In [32]:
from tensorflow.keras.layers import Input,Lambda,Dense,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob


In [33]:
ROOT_PATH=r'C:\Users\rctuh\Desktop\minor projects\Vgg16_Image_Classifier\cats_and_dogs\cats_and_dogs'

In [34]:
pwd

'C:\\Users\\rctuh\\Desktop\\minor projects\\Vgg16_Image_Classifier\\cats_and_dogs\\cats_and_dogs'

In [35]:
import os 
os.chdir(ROOT_PATH)
os.getcwd()

'C:\\Users\\rctuh\\Desktop\\minor projects\\Vgg16_Image_Classifier\\cats_and_dogs\\cats_and_dogs'

#Resizing all images

In [36]:
IMAGE_SIZE=[224,224]

In [38]:
train_path=r"C:\Users\rctuh\Desktop\minor projects\Vgg16_Image_Classifier\cats_and_dogs\cats_and_dogs\train"
test_path=r"C:\Users\rctuh\Desktop\minor projects\Vgg16_Image_Classifier\cats_and_dogs\cats_and_dogs\test"
val_path=r"C:\Users\rctuh\Desktop\minor projects\Vgg16_Image_Classifier\cats_and_dogs\cats_and_dogs\validation"

#Importing VGG16 library and add preprocesing layer in front of the VGG16.
#Here we will se ImageNet weights

In [39]:
vgg16=VGG16(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top=False)

include_top= False means that we will cut off the last dense layer of pre-trained model of VGG16 as it was trained for 1000 output categories of imagenet whereas we only have 3 categories. We also cut off the 1st layer because the input image and the size can be of my choice.

In [40]:
for layers in vgg16.layers:
  print(layers)

###Setting layer.trainable to False moves all the layer's weights from trainable to non-trainable. This is called "freezing" the layer: the state of a frozen layer won't be updated during training (either when training with fit() or when training with any custom loop that relies on trainable_weights to apply gradient updates).

In [41]:
for layer in vgg16.layers:
  layer.trainable=False

In [42]:
for layer in vgg16.layers:
  print(layer.name,layer.trainable)

input_2 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 False
block5_conv2 False
block5_conv3 False
block5_pool False


#Vgg16 Model Summary

In [43]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

#Building the Final Model 


In [44]:
model = Sequential()

model.add(vgg16)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(2,activation='Softmax'))


#View the structure of final model

In [45]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_2 (Dense)             (None, 256)               6422784   
                                                                 
 dense_3 (Dense)             (None, 2)                 514       
                                                                 
Total params: 21,137,986
Trainable params: 6,423,298
Non-trainable params: 14,714,688
_________________________________________________________________


#Specifying the Cost and Optimization Methods

In [46]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics='accuracy'
)

#Using the Image Data Generator to import images from dataset

In [47]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen= ImageDataGenerator(rescale=1./255,
                                  shear_range =0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen =  ImageDataGenerator(rescale = 1./255)

In [48]:
pwd

'C:\\Users\\rctuh\\Desktop\\minor projects\\Vgg16_Image_Classifier\\cats_and_dogs\\cats_and_dogs'

In [49]:
training_set=train_datagen.flow_from_directory(r'C:\Users\rctuh\Desktop\minor projects\Vgg16_Image_Classifier\cats_and_dogs\cats_and_dogs\train',
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode='categorical')


Found 2000 images belonging to 2 classes.


In [55]:
test_set=test_datagen.flow_from_directory(r'C:\Users\rctuh\Desktop\minor projects\Vgg16_Image_Classifier\cats_and_dogs\cats_and_dogs\test',
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode='categorical'
                                            )

Found 0 images belonging to 0 classes.


#Early stopping to avoid overfitting of model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)


#Model Fitting

In [ ]:
r= model.fit(
    training_set,
    validation_data=test_set,
    epochs=10,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set)
)

#Plotting the Losses

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(r.history['loss'], label='Train Loss')
plt.plot(r.history['val_loss'], label='Val Loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

#Plotting Accuracy 


In [ ]:
plt.plot(r.history['accuracy'], label='Train Acc')
plt.plot(r.history['val_accuracy'], label='Val Acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

#Saving the File as .h5 Type


In [ ]:
from tensorflow.keras.models import load_model 
model.save('model_vgg16.h5')

#Predictions

In [ ]:
y_pred=model.predict(test_set)


In [ ]:
y_pred

In [ ]:
import numpy as np
y_pred=np.argmax(y_pred,axis=1)

In [ ]:
y_pred

In [ ]:
from tensorflow.keras.preprocessing import image


In [ ]:
model=load_model('model_vgg16.h5')
img_path=r'C:\Users\rctuh\Desktop\minor projects\Vgg16_Image_Classifier\cats_and_dogs\cats_and_dogs\validation\dogs\dog.2012.jpg'
img=image.load_img(img_path,target_size=(224,224))


In [ ]:
x=image.img_to_array(img)

In [ ]:
Z=plt.imread(img_path)

In [ ]:
plt.imshow(Z)

In [ ]:
x.shape


In [ ]:
x=x/255

In [ ]:
from keras.applications.vgg16 import preprocess_input 

In [ ]:
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

In [ ]:
model.predict(img_data)

In [ ]:
result=np.argmax(model.predict(img_data),axis=1)

In [ ]:
result

In [ ]:
if result[0] == 1:
  prediction = "The Given Image is of a Dog"
  print(prediction)
else:
  prediction="The Given Image is of a cat"
  print(prediction)
  

Losses and Accuracy 


In [ ]:
 from sklearn.metrics import accuracy_score
 from sklearn.metrics import f1_score
 from sklearn.metrics import roc_auc_score


In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
f1_score(y_test,y_pred)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
roc_auc_score(y_test,y_pred)

In [ ]:
print(classification_report(y_pred,test_y))

In [ ]:
print(confusion_matrix(y_pred,test_y))